### 1. Query to extract client data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# df = _sqldf.toPandas()
df = pd.read_csv('/Users/rajesh/Desktop/improve_metrics_JOB.csv')

# age at first policy (calculated from dates)
df['register_date'] = pd.to_datetime(df['register_date'], errors='coerce')
df['isrd_brth_date'] = pd.to_datetime(df['isrd_brth_date'], errors='coerce')
df['age_at_first_policy'] = (df['register_date'] - df['isrd_brth_date']).dt.days / 365.25

# age at second policy
df['second_register_date'] = pd.to_datetime(df['second_register_date'], errors='coerce')
df['age_at_second_policy'] = (df['second_register_date'] - df['isrd_brth_date']).dt.days / 365.25

# time gap between first and second policy
df['years_to_second'] = (df['second_register_date'] - df['register_date']).dt.days / 365.25


In [3]:
display(df.head(5))

,axa_party_id,policy_no,register_date,trmn_eff_date,wti_lob_txt,prod_lob,agt_class,isrd_brth_date,psn_age,acct_val_amt,...,annuity_allocation_ratio,mutual_fund_allocation_ratio,aum_to_asset_ratio,policy_value_to_assets_ratio,product_category,second_product_category,season_of_first_policy,age_at_first_policy,age_at_second_policy,years_to_second
0,00BK05RY274T637VXXXX,011492461,1942-02-07,2025-02-07,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1928-11-26,97.0,0.0,...,0.031985,0.240678,0.000000,0.000235,LIFE_INSURANCE,NaN,Q1,13.199179,NaN,NaN
1,00BK05RY27EMUV5SXXXX,011995497,1945-07-19,2025-07-19,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1944-04-07,82.0,0.0,...,0.021641,0.090285,0.000000,0.021939,LIFE_INSURANCE,NaN,Q3,1.281314,NaN,NaN
2,00BK05RY27JF1H4DXXXX,012258775,1946-09-04,NaN,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1939-04-21,87.0,1818.0,...,0.043612,0.546774,0.000271,0.000298,LIFE_INSURANCE,NaN,Q3,7.373032,NaN,NaN
3,00BK05RY27L0GNKUXXXX,012370262,1947-03-21,NaN,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1947-01-16,79.0,869.0,...,NaN,NaN,NaN,NaN,LIFE_INSURANCE,NaN,Q1,0.175222,NaN,NaN
4,00BK05RY27QPLN0IXXXX,012710157,1948-06-28,NaN,Life Insurance,LIFE,DSF - DEVELOPED SALESFORCE,1940-09-02,85.0,941.0,...,0.031777,0.265780,0.002576,0.002738,LIFE_INSURANCE,RETIREMENT,Q2,7.819302,64.722793,56.903491


### **DATA CLEANING**

In [5]:
# Remove duplicate rows
df = df.drop_duplicates()

In [6]:
# Handle missing values

# drop rows with missing target or critical features
critical_cols = ['product_category']
df = df.dropna(subset=critical_cols)

num_cols = df.select_dtypes(include=['float64', 'int64']).columns
display(num_cols)

cat_cols = df.select_dtypes(include=['object']).columns
display(cat_cols)

Index(['psn_age', 'acct_val_amt', 'face_amt', 'cash_val_amt',
       'wc_total_assets', 'wc_assetmix_stocks', 'wc_assetmix_bonds',
       'wc_assetmix_mutual_funds', 'wc_assetmix_annuity',
       'wc_assetmix_deposits', 'wc_assetmix_other_assets', 'business_month',
       'branchoffice_code', 'agt_no', 'stock_allocation_ratio',
       'bond_allocation_ratio', 'annuity_allocation_ratio',
       'mutual_fund_allocation_ratio', 'aum_to_asset_ratio',
       'policy_value_to_assets_ratio', 'age_at_first_policy',
       'age_at_second_policy', 'years_to_second'],
      dtype='object')

Index(['axa_party_id', 'policy_no', 'trmn_eff_date', 'wti_lob_txt', 'prod_lob',
       'agt_class', 'client_seg', 'client_seg_1', 'aum_band',
       'sub_product_level_1', 'sub_product_level_2', 'Product',
       'division_name', 'mkt_prod_hier', 'policy_status', 'channel',
       'agent_segment', 'second_policy_no', 'second_trmn_eff_date',
       'second_wti_lob_txt', 'second_prod_lob', 'second_sub_product_level_1',
       'second_sub_product_level_2', 'second_Product', 'product_category',
       'second_product_category', 'season_of_first_policy'],
      dtype='object')

In [7]:
import scipy.stats as stats
import numpy as np

# List of financial columns
financial_cols = [col for col in df.columns if col.startswith('wc_')] + ['face_amt', 'cash_val_amt', 'acct_val_amt']
financial_cols = [col for col in financial_cols if col in df.columns]

# Compute skewness for each financial column
skewness_dict = {col: stats.skew(df[col].dropna()) for col in financial_cols}
skew_df = pd.DataFrame([skewness_dict])
display(skew_df)

# Apply log1p transformation to reduce skewness
for col in financial_cols:
    df[f'log_{col}'] = np.log1p(df[col])

# Compute skewness for each log-transformed financial column
log_skewness_dict = {f'log_{col}': stats.skew(df[f'log_{col}'].dropna()) for col in financial_cols}
log_skew_df = pd.DataFrame([log_skewness_dict])
display(log_skew_df)

,wc_total_assets,wc_assetmix_stocks,wc_assetmix_bonds,wc_assetmix_mutual_funds,wc_assetmix_annuity,wc_assetmix_deposits,wc_assetmix_other_assets,face_amt,cash_val_amt,acct_val_amt
0,3.299423,3.949428,7.215579,3.776952,4.721003,6.720008,9.29926,17.432612,11.072645,8.773947


/Users/rajesh/.pyenv/versions/3.11.6/envs/my-vscode-env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


,log_wc_total_assets,log_wc_assetmix_stocks,log_wc_assetmix_bonds,log_wc_assetmix_mutual_funds,log_wc_assetmix_annuity,log_wc_assetmix_deposits,log_wc_assetmix_other_assets,log_face_amt,log_cash_val_amt,log_acct_val_amt
0,-3.252003,-1.65398,-1.124151,-2.121249,-2.238858,-5.091397,-1.165425,-1.503723,0.535675,-0.563845


In [8]:
# Standardize date columns
date_cols = ['register_date', 'second_register_date', 'isrd_brth_date', 'trmn_eff_date', 'second_trmn_eff_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Remove outliers in numerical features
df = df[(df['age_at_first_policy'] >= 0) & (df['age_at_first_policy'] <= 100)]

# Categorical encoding (LabelEncoder is correct for tree models, but for Spark MLlib, use StringIndexer)
cat_cols = [
    'product_category', 'prod_lob', 'client_seg', 'aum_band', 'agt_class', 'season_of_first_policy', 'client_seg_1', 'division_name','mkt_prod_hier', 'policy_status', 'channel', 'agent_segment']
for col in cat_cols + ['second_product_category']:
    if col in df.columns:
        df[col] = df[col].astype(str)


In [11]:
# Target encoding with 5-fold regularization for high-cardinality categoricals
from sklearn.model_selection import KFold
import numpy as np

high_cardinality_cols = [
    # "agt_no",
    # "branchoffice_code",
    "client_seg",
    "product_category",
    "premium_payment_method",
    "aum_band",
    "channel"
]
high_cardinality_cols = [col for col in high_cardinality_cols if col in df.columns]
target_col = "second_product_category"

if high_cardinality_cols:
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    df["_target_idx"] = df[target_col].astype("category").cat.codes
    valid_indices = df.index[df[target_col].notnull()].to_numpy()
    global_mean = df.loc[df[target_col].notnull(), "_target_idx"].mean()
    smoothing = 20

    for col in high_cardinality_cols:
        te_col = f"te_{col}"
        df[te_col] = np.nan
        for train_pos, val_pos in kfold.split(valid_indices):
            train_idx = valid_indices[train_pos]
            val_idx = valid_indices[val_pos]
            fold_stats = df.loc[train_idx].groupby(col)["_target_idx"].agg(["mean", "count"])
            fold_stats["smoothed"] = (
                fold_stats["mean"] * fold_stats["count"] + global_mean * smoothing
            ) / (fold_stats["count"] + smoothing)
            mapping = fold_stats["smoothed"]
            df.loc[val_idx, te_col] = df.loc[val_idx, col].map(mapping)
        df[te_col] = df[te_col].fillna(global_mean)
    df.drop(columns=["_target_idx"], inplace=True)
else:
    print("No high-cardinality columns available for target encoding.")


In [12]:
df.head()

,axa_party_id,policy_no,register_date,trmn_eff_date,wti_lob_txt,prod_lob,agt_class,isrd_brth_date,psn_age,acct_val_amt,...,log_wc_assetmix_other_assets,log_face_amt,log_cash_val_amt,log_acct_val_amt,te_client_seg,te_product_category,te_aum_band,te_channel,te_agt_no,te_branchoffice_code
0,00BK05RY274T637VXXXX,011492461,1942-02-07,2025-02-07,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1928-11-26,97.0,0.0,...,12.265548,6.908755,0.000000,0.000000,4.500928,4.487555,4.636932,4.559936,4.632931,4.672804
1,00BK05RY27EMUV5SXXXX,011995497,1945-07-19,2025-07-19,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1944-04-07,82.0,0.0,...,6.190315,7.131699,0.000000,0.000000,4.708777,4.485959,4.643772,4.563796,4.529938,4.656731
2,00BK05RY27JF1H4DXXXX,012258775,1946-09-04,NaT,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1939-04-21,87.0,1818.0,...,11.097061,7.601402,7.506042,7.506042,4.627197,4.494117,4.638026,4.664246,4.666301,4.391090
3,00BK05RY27L0GNKUXXXX,012370262,1947-03-21,NaT,Life Insurance,LIFE,ESF - EXPERIENCED SALESFORCE,1947-01-16,79.0,869.0,...,0.000000,6.908755,6.768493,6.768493,4.650811,4.484090,4.628955,4.669040,4.669040,4.669040
4,00BK05RY27QPLN0IXXXX,012710157,1948-06-28,NaT,Life Insurance,LIFE,DSF - DEVELOPED SALESFORCE,1940-09-02,85.0,941.0,...,6.261492,6.908755,6.848005,6.848005,4.625456,4.485959,4.643772,4.563796,4.632931,4.529708


In [13]:
# Interaction features to capture non-linear relationships
if {"age_at_first_policy", "wc_total_assets"}.issubset(df.columns):
    df["age_assets"] = df["age_at_first_policy"] * df["wc_total_assets"]
else:
    df["age_assets"] = np.nan

if {"age_at_first_policy", "stock_allocation_ratio"}.issubset(df.columns):
    df["age_equity_ratio"] = df["age_at_first_policy"] * df["stock_allocation_ratio"]
else:
    df["age_equity_ratio"] = np.nan

if "register_date" in df.columns:
    reference_date = df["register_date"].max()
    df["days_since_first_policy"] = (reference_date - df["register_date"]).dt.days
else:
    df["days_since_first_policy"] = np.nan

if "wc_total_assets" in df.columns:
    df["log_total_assets"] = np.log1p(df["wc_total_assets"].clip(lower=0))
else:
    df["log_total_assets"] = np.nan

if {"stock_allocation_ratio", "bond_allocation_ratio"}.issubset(df.columns):
    df["equity_to_bond_ratio"] = df["stock_allocation_ratio"] / df["bond_allocation_ratio"].replace(0, np.nan)
else:
    df["equity_to_bond_ratio"] = np.nan

if {"premium_amount", "income_estimate"}.issubset(df.columns):
    df["premium_to_income"] = df["premium_amount"] / df["income_estimate"].replace(0, np.nan)
else:
    df["premium_to_income"] = np.nan


In [14]:
# Customer clustering on standardized behavioral features
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

cluster_features = [
    "age_at_first_policy",
    "wc_total_assets",
    "stock_allocation_ratio",
    "bond_allocation_ratio",
    "annuity_allocation_ratio",
    "mutual_fund_allocation_ratio",
    "aum_to_asset_ratio",
    "policy_value_to_assets_ratio",
    "age_assets",
    "age_equity_ratio",
    "log_total_assets",
    "equity_to_bond_ratio",
    "premium_to_income",
    "days_since_first_policy"
]
available_cluster_features = [col for col in cluster_features if col in df.columns]

if available_cluster_features:
    scaler = StandardScaler()
    cluster_input = df[available_cluster_features].fillna(0)
    scaled_features = scaler.fit_transform(cluster_input)
    kmeans = KMeans(n_clusters=8, random_state=42, n_init=10)
    df["client_cluster"] = kmeans.fit_predict(scaled_features).astype(str)
else:
    df["client_cluster"] = "UNKNOWN"

if "client_cluster" not in cat_cols:
    cat_cols = list(cat_cols) + ["client_cluster"]


/Users/rajesh/.pyenv/versions/3.11.6/envs/my-vscode-env/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/rajesh/.pyenv/versions/3.11.6/envs/my-vscode-env/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/rajesh/.pyenv/versions/3.11.6/envs/my-vscode-env/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/rajesh/.pyenv/versions/3.11.6/envs/my-vscode-env/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/rajesh/.pyenv/versions/3.11.6/envs/my-vscode-env/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: overflow encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/rajesh/.pyenv/versions/3

In [15]:
df.columns

Index(['axa_party_id', 'policy_no', 'register_date', 'trmn_eff_date',
       'wti_lob_txt', 'prod_lob', 'agt_class', 'isrd_brth_date', 'psn_age',
       'acct_val_amt', 'face_amt', 'cash_val_amt', 'wc_total_assets',
       'wc_assetmix_stocks', 'wc_assetmix_bonds', 'wc_assetmix_mutual_funds',
       'wc_assetmix_annuity', 'wc_assetmix_deposits',
       'wc_assetmix_other_assets', 'client_seg', 'client_seg_1', 'aum_band',
       'sub_product_level_1', 'sub_product_level_2', 'Product',
       'business_month', 'branchoffice_code', 'agt_no', 'division_name',
       'mkt_prod_hier', 'policy_status', 'channel', 'agent_segment',
       'second_policy_no', 'second_register_date', 'second_trmn_eff_date',
       'second_wti_lob_txt', 'second_prod_lob', 'second_sub_product_level_1',
       'second_sub_product_level_2', 'second_Product',
       'stock_allocation_ratio', 'bond_allocation_ratio',
       'annuity_allocation_ratio', 'mutual_fund_allocation_ratio',
       'aum_to_asset_ratio', 'policy

In [16]:
df = df.drop(columns=[
    'log_wc_assetmix_stocks',
    'log_wc_assetmix_bonds',
    'log_wc_assetmix_mutual_funds',
    'log_wc_assetmix_deposits',
    'log_wc_assetmix_other_assets',
    'log_acct_val_amt'
])

In [17]:
df.columns

Index(['axa_party_id', 'policy_no', 'register_date', 'trmn_eff_date',
       'wti_lob_txt', 'prod_lob', 'agt_class', 'isrd_brth_date', 'psn_age',
       'acct_val_amt', 'face_amt', 'cash_val_amt', 'wc_total_assets',
       'wc_assetmix_stocks', 'wc_assetmix_bonds', 'wc_assetmix_mutual_funds',
       'wc_assetmix_annuity', 'wc_assetmix_deposits',
       'wc_assetmix_other_assets', 'client_seg', 'client_seg_1', 'aum_band',
       'sub_product_level_1', 'sub_product_level_2', 'Product',
       'business_month', 'branchoffice_code', 'agt_no', 'division_name',
       'mkt_prod_hier', 'policy_status', 'channel', 'agent_segment',
       'second_policy_no', 'second_register_date', 'second_trmn_eff_date',
       'second_wti_lob_txt', 'second_prod_lob', 'second_sub_product_level_1',
       'second_sub_product_level_2', 'second_Product',
       'stock_allocation_ratio', 'bond_allocation_ratio',
       'annuity_allocation_ratio', 'mutual_fund_allocation_ratio',
       'aum_to_asset_ratio', 'policy

In [19]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 27.6 MB/s  0:00:16:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813860 sha256=e83242738cf2d24be9190b45f212063b0ead20b2dbb438f3c0a792127452d9e6
  Stored in directory: /Users/rajesh/Library/Caches/pip/wheels/76/f8/dc/9195b82b8586561710077d42370ae400e8a023af43052d1fec
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pyspark]m1/2 [pyspark]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [22]:
!java -version


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("eq-mpe") \
    .master("local[*]") \
    .getOrCreate()


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/rajesh/.pyenv/versions/3.11.6/envs/my-vscode-env/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [20]:
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Median imputation for allocation ratio columns
allocation_cols = [
    'stock_allocation_ratio', 'bond_allocation_ratio', 'annuity_allocation_ratio',
    'mutual_fund_allocation_ratio', 'aum_to_asset_ratio', 'policy_value_to_assets_ratio'
]

if 'spark' not in globals() or spark is None:
    raise RuntimeError("Spark context is not available. Please run this cell in a Databricks notebook with an active cluster.")

if not hasattr(df, 'columns'):
    raise RuntimeError("df is not defined or not a pandas DataFrame.")

imputer = Imputer(
    inputCols=[c for c in allocation_cols if c in df.columns],
    outputCols=[c for c in allocation_cols if c in df.columns],
    strategy="median"
)
spark_df = spark.createDataFrame(df)
df_imputed = imputer.fit(spark_df).transform(spark_df)
spark_df = df_imputed
display(spark_df)

RuntimeError: Spark context is not available. Please run this cell in a Databricks notebook with an active cluster.

In [0]:
def add_propensity_features(df, lookup_prod, lookup_mode, lookup_agent, lookup_branch):
    df_with_features = df.withColumnRenamed("product_category", "prod_code")
    df_with_features = df_with_features.join(lookup_prod, on="prod_code", how="left")
    df_with_features = df_with_features.join(lookup_mode, on="prod_code", how="left")
    df_with_features = df_with_features.join(lookup_agent, on=["agt_no", "prod_code"], how="left")
    df_with_features = df_with_features.join(lookup_branch, on=["branchoffice_code", "prod_code"], how="left")
    df_with_features = df_with_features.na.fill(0, [
        "p1_cross_sell_popularity",
        "agent_p1_cross_sell_count",
        "branch_p1_cross_sell_count"
    ])
    df_with_features = df_with_features.na.fill("UNKNOWN", ["p1_most_common_next_prod"])
    df_with_features = df_with_features.withColumnRenamed("prod_code", "product_category")
    return df_with_features

In [0]:
log_financial_cols = [
    'log_wc_total_assets', 'log_wc_assetmix_annuity', 'log_wc_assetmix_other_assets',
    'log_face_amt', 'log_cash_val_amt'
]

# Train only on rows with second_product_category (i.e., with a second policy)
train_df = spark_df.filter(col("second_product_category").isNotNull())
for colname in ['age_at_first_policy', 'years_to_second'] + [col for col in log_financial_cols if col in df.columns]:
    train_df = train_df.filter(col(colname).isNotNull())

In [0]:
# Split train_df into training and validation sets
train_data, val_data = train_df.randomSplit([0.8, 0.2], seed=42)

In [0]:
# Defining a new df for creating propensity features

cross_sell_history_df = train_data.select(
    F.col("product_category").alias("first_prod_code"),
    F.col("second_product_category").alias("second_prod_code"),
    "agt_no",
    "branchoffice_code"
)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [ ]:
# Placeholder cell: class weights are computed after propensity features are added.


In [0]:
# Feature: Most common next product (mode)
prod_total_cross_sells = cross_sell_history_df.groupBy("first_prod_code") \
    .count() \
    .withColumnRenamed("count", "p1_total_cross_sells")
propensity_prod_df = prod_total_cross_sells.withColumnRenamed("first_prod_code", "prod_code") \
    .withColumnRenamed("p1_total_cross_sells", "p1_cross_sell_popularity")
display(propensity_prod_df)

path_counts = cross_sell_history_df.groupBy("first_prod_code", "second_prod_code").count()
window_spec = Window.partitionBy("first_prod_code").orderBy(F.col("count").desc())
most_common_path_df = path_counts.withColumn("rank", F.row_number().over(window_spec)) \
    .filter(F.col("rank") == 1) \
    .select(
        F.col("first_prod_code").alias("prod_code"),
        F.col("second_prod_code").alias("p1_most_common_next_prod")
    )
display(most_common_path_df)

# Feature: Agent-level cross-sell count
propensity_agent_df = cross_sell_history_df.groupBy("agt_no", "first_prod_code") \
    .count() \
    .withColumnRenamed("count", "agent_p1_cross_sell_count") \
    .withColumnRenamed("first_prod_code", "prod_code")
display(propensity_agent_df)

# Feature: Branch-level cross-sell count
propensity_branch_df = cross_sell_history_df.groupBy("branchoffice_code", "first_prod_code") \
    .count() \
    .withColumnRenamed("count", "branch_p1_cross_sell_count") \
    .withColumnRenamed("first_prod_code", "prod_code")
display(propensity_branch_df)

In [0]:
# Add propensity features to train and validation sets
train_df_final = add_propensity_features(
    train_data, propensity_prod_df, most_common_path_df, propensity_agent_df, propensity_branch_df
)
val_df_final = add_propensity_features(
    val_data, propensity_prod_df, most_common_path_df, propensity_agent_df, propensity_branch_df
)

In [ ]:
# Compute per-class weights to balance the training objective
class_counts = train_df_final.groupBy("second_product_category").count()
total_samples = train_df_final.count()
num_classes = class_counts.count()

class_weights = class_counts.withColumn(
    "class_weight",
    (F.lit(total_samples) / (F.lit(num_classes) * F.col("count")))
)

weight_cols = class_weights.select("second_product_category", "class_weight")
train_df_final = train_df_final.join(weight_cols, on="second_product_category", how="left")
val_df_final = val_df_final.join(weight_cols, on="second_product_category", how="left").fillna({"class_weight": 1.0})


Gather columns for training

In [0]:
# Assemble feature lists for CatBoost
cat_cols = list(set(cat_cols + ['p1_most_common_next_prod', 'client_cluster']))

target_encoded_cols = [col for col in df.columns if col.startswith("te_")]
interaction_cols = [
    "age_assets",
    "age_equity_ratio",
    "days_since_first_policy",
    "log_total_assets",
    "equity_to_bond_ratio",
    "premium_to_income"
]
cluster_cols = ["client_cluster"]
ratio_cols = [
    'stock_allocation_ratio', 'bond_allocation_ratio', 'annuity_allocation_ratio',
    'mutual_fund_allocation_ratio', 'aum_to_asset_ratio', 'policy_value_to_assets_ratio'
]
base_numeric_cols = ['age_at_first_policy', 'years_to_second'] + [col for col in log_financial_cols if col in df.columns]
propensity_cols = [
    "p1_cross_sell_popularity",
    "agent_p1_cross_sell_count",
    "branch_p1_cross_sell_count"
]

feature_candidates = set(
    target_encoded_cols + interaction_cols + cluster_cols + ratio_cols + base_numeric_cols + propensity_cols
)
feature_cols = [col for col in feature_candidates if col in train_df_final.columns]

categorical_feature_cols = [col for col in cat_cols if col in feature_cols]
numeric_feature_cols = [col for col in feature_cols if col not in categorical_feature_cols]
feature_cols = categorical_feature_cols + numeric_feature_cols
print(f"Using {len(feature_cols)} features ({len(categorical_feature_cols)} categorical / {len(numeric_feature_cols)} numeric).")

In [0]:
# Ensure CatBoost is available and import evaluation helpers
import importlib
import subprocess
import sys

try:
    import catboost  # noqa: F401
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "catboost"])

from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, f1_score

In [0]:
# Convert Spark DataFrames to pandas for CatBoost
id_cols = ["axa_party_id", "policy_no", "product_category"]
available_id_cols = [col for col in id_cols if col in train_df_final.columns]
selected_cols = feature_cols + available_id_cols + ["second_product_category", "class_weight"]
train_pd = train_df_final.select(*selected_cols).toPandas()
val_pd = val_df_final.select(*selected_cols).toPandas()

cat_feature_names = [col for col in categorical_feature_cols if col in train_pd.columns]

train_pool = Pool(
    data=train_pd[feature_cols],
    label=train_pd["second_product_category"],
    cat_features=cat_feature_names,
    weight=train_pd["class_weight"],
)

val_pool = Pool(
    data=val_pd[feature_cols],
    label=val_pd["second_product_category"],
    cat_features=cat_feature_names,
    weight=val_pd["class_weight"],
)

In [0]:
### CATBOOST TRAINING

cat_model = CatBoostClassifier(
    iterations=800,
    depth=9,
    learning_rate=0.05,
    loss_function="MultiClass",
    eval_metric="TotalF1",
    random_seed=42,
    task_type="CPU",
    l2_leaf_reg=3.0,
    subsample=0.85,
    colsample_bylevel=0.8,
    auto_class_weights=None
)
cat_model.fit(train_pool, eval_set=val_pool, verbose=100)

In [0]:
# Evaluate CatBoost on the validation set
val_predictions = cat_model.predict(val_pool)
val_probabilities = cat_model.predict_proba(val_pool)

f1_macro = f1_score(val_pd["second_product_category"], val_predictions, average="macro")
print(f"Validation Macro F1: {f1_macro:.4f}")
print(classification_report(val_pd["second_product_category"], val_predictions))

predictions_df = val_pd.copy()
predictions_df["predicted_second_product_category"] = val_predictions
predictions_df["max_probability"] = val_probabilities.max(axis=1)
display(predictions_df[["axa_party_id", "policy_no", "product_category", "second_product_category", "predicted_second_product_category", "max_probability"]])

In [ ]:
# Feature importance from CatBoost
import pandas as pd
importance = cat_model.get_feature_importance(type="FeatureImportance")
feature_importance = pd.DataFrame({
    "feature": feature_cols,
    "importance": importance
}).sort_values("importance", ascending=False)
display(feature_importance)


Storing the Propensity features in the catalog explorer for pipeline preprocessing

In [0]:
propensity_prod_df.write.format("delta").mode("overwrite").saveAsTable("eda_smartlist.us_wealth_management_smartlist.propensity_prod_df")

most_common_path_df.write.format("delta").mode("overwrite").saveAsTable("eda_smartlist.us_wealth_management_smartlist.most_common_path_df")

propensity_agent_df.write.format("delta").mode("overwrite").saveAsTable("eda_smartlist.us_wealth_management_smartlist.propensity_agent_df")

propensity_branch_df.write.format("delta").mode("overwrite").saveAsTable("eda_smartlist.us_wealth_management_smartlist.propensity_branch_df")

Registering the model in the catalog explorer

In [0]:
import mlflow
import mlflow.catboost
from mlflow.models import infer_signature

input_example = train_pd[feature_cols].head(5)
output_example = cat_model.predict_proba(train_pd[feature_cols].head(5))
signature = infer_signature(input_example, output_example)

with mlflow.start_run() as run:
    mlflow.log_metric("val_macro_f1", f1_macro)
    model_info = mlflow.catboost.log_model(
        cb_model=cat_model,
        artifact_path="catboost_propfeat_model",
        registered_model_name="eda_smartlist.models.catboost_propfeat_model",
        signature=signature,
        input_example=input_example
    )
catboost_model_version = model_info.model_version

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

Evaluating the CatBoost model using Accuracy, Precision, Recall, and Macro F1

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(val_pd["second_product_category"], val_predictions)
precision = precision_score(val_pd["second_product_category"], val_predictions, average="macro")
recall = recall_score(val_pd["second_product_category"], val_predictions, average="macro")

metrics_df = pd.DataFrame([
    {"accuracy": accuracy, "precision_macro": precision, "recall_macro": recall, "f1_macro": f1_macro}
])
display(metrics_df)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

In [0]:
# Register and create/update an alias for the CatBoost model
from mlflow import MlflowClient

client = MlflowClient()
model_name = "eda_smartlist.models.catboost_propfeat_model"
alias = "catboost_propfeat_prod"

if catboost_model_version:
    client.set_registered_model_alias(
        name=model_name,
        alias=alias,
        version=catboost_model_version
    )
else:
    print("Model version unavailable; skipping alias assignment.")

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:473)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

Hyperparameter tuning for CatBoost

In [0]:
from itertools import product

param_grid = {
    "depth": [8, 9, 10],
    "learning_rate": [0.03, 0.05],
    "l2_leaf_reg": [3, 5],
    "subsample": [0.8, 0.9]
}

best_params = None
best_f1 = -1
search_results = []

for depth, lr, l2, subsample in product(
    param_grid["depth"],
    param_grid["learning_rate"],
    param_grid["l2_leaf_reg"],
    param_grid["subsample"]
):
    params = dict(depth=depth, learning_rate=lr, l2_leaf_reg=l2, subsample=subsample)
    tuner_model = CatBoostClassifier(
        iterations=500,
        loss_function="MultiClass",
        eval_metric="TotalF1",
        random_seed=42,
        task_type="CPU",
        verbose=False,
        **params
    )
    tuner_model.fit(train_pool, eval_set=val_pool, verbose=False)
    preds = tuner_model.predict(val_pool)
    score = f1_score(val_pd["second_product_category"], preds, average="macro")
    search_results.append({**params, "f1_macro": score})
    if score > best_f1:
        best_f1 = score
        best_params = params

search_results_df = pd.DataFrame(search_results).sort_values("f1_macro", ascending=False)
display(search_results_df)
print("Best params:", best_params, "with macro F1", best_f1)

In [0]:
# Optionally refit CatBoost with the best params discovered above
if best_params is not None:
    tuned_model = CatBoostClassifier(
        iterations=800,
        loss_function="MultiClass",
        eval_metric="TotalF1",
        random_seed=42,
        task_type="CPU",
        verbose=100,
        **best_params
    )
    tuned_model.fit(train_pool, eval_set=val_pool)
    tuned_preds = tuned_model.predict(val_pool)
    tuned_f1 = f1_score(val_pd["second_product_category"], tuned_preds, average="macro")
    print(f"Tuned model Macro F1: {tuned_f1:.4f}")
else:
    print("No best params identified; skipping tuned refit.")

### **Next Steps**
1. Launch full CatBoost hyperparameter sweep (e.g., Optuna/HypOpt) and log the best run.
2. Calibrate decision thresholds per class to maximize macro F1 on the validation set.
3. Productionize the preprocessing stack (target encoding, clustering artifacts, CatBoost model) behind a reproducible pipeline/notebook job.